**Задание**

1. Использовать датасет MovieLens.

2. Построить рекомендации (регрессия, предсказываем оценку) на фичах:
 - TF-IDF на тегах и жанрах;

 - средние оценки (+ median, variance и т. д.) пользователя и фильма.

3. Оценить RMSE на тестовой выборке.

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

Загрузим данные

In [ ]:
!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens.zip

--2025-01-02 18:37:21--  https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.google.com (drive.google.com)... 74.125.137.102, 74.125.137.101, 74.125.137.113, ...
Connecting to drive.google.com (drive.google.com)|74.125.137.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx [following]
--2025-01-02 18:37:21--  https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.137.132, 2607:f8b0:4023:c03::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.137.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 847695 (828K) [application/octet-stream]
Saving to: ‘MovieLens.zip’

MovieLens.zip       100%[===================>] 827.83K  --.-KB/s    in 0.1s    

2025-01-02 18:37:24 (6.99 MB/s) -

Разархивируем данные

In [ ]:
!unzip MovieLens.zip

Archive:  MovieLens.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: tags.csv                


In [ ]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [ ]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [ ]:
tags['tag'].count()

3683

In [ ]:
tags['tag'].value_counts()

,count
tag,
In Netflix queue,131
atmospheric,36
thought-provoking,24
superhero,24
funny,23
...,...
small towns,1
In Your Eyes,1
Lloyd Dobbler,1


In [ ]:
tags['tag'].unique()

array(['funny', 'Highly quotable', 'will ferrell', ..., 'gun fu',
       'heroic bloodshed', 'Heroic Bloodshed'], dtype=object)

In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
ratings['rating'].value_counts()

,count
rating,
4.0,26818
3.0,20047
5.0,13211
3.5,13136
4.5,8551
2.0,7551
2.5,5550
1.0,2811
1.5,1791


In [ ]:
ratings['userId'].count()

100836

In [ ]:
ratings['movieId'].count()

100836

In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
movies['title'].value_counts()

,count
title,
Emma (1996),2
War of the Worlds (2005),2
Confessions of a Dangerous Mind (2002),2
Eros (2004),2
Saturn 3 (1980),2
...,...
Lost and Delirious (2001),1
Rape Me (Baise-moi) (2000),1
Alice (1990),1


In [ ]:
movies['genres'].value_counts()

,count
genres,
Drama,1053
Comedy,946
Comedy|Drama,435
Comedy|Romance,363
Drama|Romance,349
...,...
Action|Crime|Horror|Mystery|Thriller,1
Adventure|Animation|Children|Comedy|Musical|Romance,1
Action|Adventure|Animation|Comedy|Crime|Mystery,1


In [ ]:
movies['movieId'].count()

9742

In [ ]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [ ]:
links['movieId'].count()

9742

In [ ]:
ratings.shape # число строк и столбцов

(100836, 4)

In [ ]:
movies.shape # число строк и столбцов

(9742, 3)

In [ ]:
tags.shape # число строк и столбцов

(3683, 4)

In [ ]:
links.shape # число строк и столбцов

(9742, 3)

Преобразуем дату

In [ ]:
ratings['dt'] = ratings['timestamp'].apply(lambda t: datetime.fromtimestamp(t))

ratings['year'] = ratings['dt'].dt.year
ratings['month'] = ratings['dt'].dt.month

ratings.head()

,userId,movieId,rating,timestamp,dt,year,month
0,1,1,4.0,964982703,2000-07-30 18:45:03,2000,7
1,1,3,4.0,964981247,2000-07-30 18:20:47,2000,7
2,1,6,4.0,964982224,2000-07-30 18:37:04,2000,7
3,1,47,5.0,964983815,2000-07-30 19:03:35,2000,7
4,1,50,5.0,964982931,2000-07-30 18:48:51,2000,7


In [ ]:
pd.value_counts(ratings.dtypes) # посмотрим на тип данных

<ipython-input-21-6e8be05a97f6>:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(ratings.dtypes) # посмотрим на тип данных


,count
int64,3
int32,2
float64,1
datetime64[ns],1


In [ ]:
ratings = ratings.drop(['timestamp', 'dt'], axis=1)
ratings

,userId,movieId,rating,year,month
0,1,1,4.0,2000,7
1,1,3,4.0,2000,7
2,1,6,4.0,2000,7
3,1,47,5.0,2000,7
4,1,50,5.0,2000,7
...,...,...,...,...,...
100831,610,166534,4.0,2017,5
100832,610,168248,5.0,2017,5
100833,610,168250,5.0,2017,5
100834,610,168252,5.0,2017,5


Объединим данные из таблиц

In [ ]:
data = ratings.merge(movies, how = 'left', on = 'movieId')
data

,userId,movieId,rating,year,month,title,genres
0,1,1,4.0,2000,7,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000,7,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000,7,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000,7,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000,7,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...,...,...
100831,610,166534,4.0,2017,5,Split (2017),Drama|Horror|Thriller
100832,610,168248,5.0,2017,5,John Wick: Chapter Two (2017),Action|Crime|Thriller
100833,610,168250,5.0,2017,5,Get Out (2017),Horror
100834,610,168252,5.0,2017,5,Logan (2017),Action|Sci-Fi


In [ ]:
tags = tags.drop(['userId', 'timestamp'], axis=1)
tags

,movieId,tag
0,60756,funny
1,60756,Highly quotable
2,60756,will ferrell
3,89774,Boxing story
4,89774,MMA
...,...,...
3678,7382,for katie
3679,7936,austere
3680,3265,gun fu
3681,3265,heroic bloodshed


In [ ]:
data = data.merge(tags, how = 'left', on = 'movieId')
data

,userId,movieId,rating,year,month,title,genres,tag
0,1,1,4.0,2000,7,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar
1,1,1,4.0,2000,7,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar
2,1,1,4.0,2000,7,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,fun
3,1,3,4.0,2000,7,Grumpier Old Men (1995),Comedy|Romance,moldy
4,1,3,4.0,2000,7,Grumpier Old Men (1995),Comedy|Romance,old
...,...,...,...,...,...,...,...,...
285757,610,168252,5.0,2017,5,Logan (2017),Action|Sci-Fi,emotional
285758,610,168252,5.0,2017,5,Logan (2017),Action|Sci-Fi,gritty
285759,610,168252,5.0,2017,5,Logan (2017),Action|Sci-Fi,heartbreaking
285760,610,168252,5.0,2017,5,Logan (2017),Action|Sci-Fi,predictible plot


Вычислим средние и медианные оценки пользователей и фильмов

In [ ]:
data.groupby('userId').rating.agg(rating_mean_user = ('mean'), rating_median_user = ('median')).round(2)#.apply(list).to_dict()

,rating_mean_user,rating_median_user
userId,,
1,4.09,4.0
2,3.99,4.0
3,1.98,0.5
4,2.80,3.0
5,4.49,5.0
...,...,...
606,3.93,4.0
607,3.58,3.0
608,3.58,4.0


In [ ]:
data.groupby('title').rating.agg(rating_mean_film = ('mean'), rating_median_film = ('median')).round(2)

,rating_mean_film,rating_median_film
title,,
'71 (2014),4.00,4.0
'Hellboy': The Seeds of Creation (2004),4.00,4.0
'Round Midnight (1986),3.50,3.5
'Salem's Lot (2004),5.00,5.0
'Til There Was You (1997),4.00,4.0
...,...,...
eXistenZ (1999),3.86,4.0
xXx (2002),2.77,3.0
xXx: State of the Union (2005),2.00,2.0


Создадим словари для дальнейшего заполнения столбцов датасета средними значениями и медианными значениями

In [ ]:
user_mean = data.groupby('userId').rating.agg(rating_mean_user = ('median')).round(2).apply(list).to_dict()
user_mean

{'rating_mean_user': {1: 4.0,
  2: 4.0,
  3: 0.5,
  4: 3.0,
  5: 5.0,
  6: 3.0,
  7: 4.0,
  8: 4.0,
  9: 4.0,
  10: 1.0,
  11: 4.0,
  12: 5.0,
  13: 4.0,
  14: 3.0,
  15: 4.0,
  16: 3.5,
  17: 4.5,
  18: 4.0,
  19: 3.0,
  20: 4.0,
  21: 3.5,
  22: 3.0,
  23: 3.5,
  24: 4.0,
  25: 5.0,
  26: 4.0,
  27: 4.0,
  28: 3.5,
  29: 4.0,
  30: 5.0,
  31: 4.0,
  32: 5.0,
  33: 5.0,
  34: 4.0,
  35: 4.0,
  36: 2.75,
  37: 5.0,
  38: 3.0,
  39: 5.0,
  40: 5.0,
  41: 4.0,
  42: 4.0,
  43: 3.0,
  44: 4.0,
  45: 5.0,
  46: 4.0,
  47: 2.5,
  48: 4.0,
  49: 4.5,
  50: 3.0,
  51: 4.5,
  52: 4.5,
  53: 5.0,
  54: 3.0,
  55: 4.0,
  56: 4.0,
  57: 4.0,
  58: 5.0,
  59: 5.0,
  60: 4.0,
  61: 4.5,
  62: 4.5,
  63: 4.0,
  64: 4.0,
  65: 4.5,
  66: 4.5,
  67: 4.5,
  68: 3.0,
  69: 5.0,
  70: 4.5,
  71: 3.0,
  72: 4.5,
  73: 4.0,
  74: 5.0,
  75: 4.0,
  76: 4.0,
  77: 2.5,
  78: 4.0,
  79: 5.0,
  80: 4.5,
  81: 3.0,
  82: 3.5,
  83: 4.5,
  84: 4.0,
  85: 5.0,
  86: 4.0,
  87: 3.75,
  88: 4.5,
  89: 3.5,
  90: 5.

In [ ]:
user_median = data.groupby('userId').rating.agg(rating_median_user = ('median')).round(2).apply(list).to_dict()
user_median

{'rating_median_user': {1: 4.0,
  2: 4.0,
  3: 0.5,
  4: 3.0,
  5: 5.0,
  6: 3.0,
  7: 4.0,
  8: 4.0,
  9: 4.0,
  10: 1.0,
  11: 4.0,
  12: 5.0,
  13: 4.0,
  14: 3.0,
  15: 4.0,
  16: 3.5,
  17: 4.5,
  18: 4.0,
  19: 3.0,
  20: 4.0,
  21: 3.5,
  22: 3.0,
  23: 3.5,
  24: 4.0,
  25: 5.0,
  26: 4.0,
  27: 4.0,
  28: 3.5,
  29: 4.0,
  30: 5.0,
  31: 4.0,
  32: 5.0,
  33: 5.0,
  34: 4.0,
  35: 4.0,
  36: 2.75,
  37: 5.0,
  38: 3.0,
  39: 5.0,
  40: 5.0,
  41: 4.0,
  42: 4.0,
  43: 3.0,
  44: 4.0,
  45: 5.0,
  46: 4.0,
  47: 2.5,
  48: 4.0,
  49: 4.5,
  50: 3.0,
  51: 4.5,
  52: 4.5,
  53: 5.0,
  54: 3.0,
  55: 4.0,
  56: 4.0,
  57: 4.0,
  58: 5.0,
  59: 5.0,
  60: 4.0,
  61: 4.5,
  62: 4.5,
  63: 4.0,
  64: 4.0,
  65: 4.5,
  66: 4.5,
  67: 4.5,
  68: 3.0,
  69: 5.0,
  70: 4.5,
  71: 3.0,
  72: 4.5,
  73: 4.0,
  74: 5.0,
  75: 4.0,
  76: 4.0,
  77: 2.5,
  78: 4.0,
  79: 5.0,
  80: 4.5,
  81: 3.0,
  82: 3.5,
  83: 4.5,
  84: 4.0,
  85: 5.0,
  86: 4.0,
  87: 3.75,
  88: 4.5,
  89: 3.5,
  90: 

In [ ]:
film_mean = data.groupby('title').rating.agg(rating_mean_film = ('mean')).round(2).apply(list).to_dict()
film_mean

{'rating_mean_film': {"'71 (2014)": 4.0,
  "'Hellboy': The Seeds of Creation (2004)": 4.0,
  "'Round Midnight (1986)": 3.5,
  "'Salem's Lot (2004)": 5.0,
  "'Til There Was You (1997)": 4.0,
  "'Tis the Season for Love (2015)": 1.5,
  "'burbs, The (1989)": 3.18,
  "'night Mother (1986)": 3.0,
  '(500) Days of Summer (2009)': 3.67,
  '*batteries not included (1987)': 3.29,
  '...All the Marbles (1981)': 2.0,
  '...And Justice for All (1979)': 3.17,
  '00 Schneider - Jagd auf Nihil Baxter (1994)': 4.5,
  '1-900 (06) (1994)': 4.0,
  '10 (1979)': 3.38,
  '10 Cent Pistol (2015)': 1.25,
  '10 Cloverfield Lane (2016)': 3.68,
  '10 Items or Less (2006)': 2.67,
  '10 Things I Hate About You (1999)': 3.53,
  '10 Years (2011)': 3.5,
  '10,000 BC (2008)': 2.71,
  '100 Girls (2000)': 3.25,
  '100 Streets (2016)': 2.5,
  '101 Dalmatians (1996)': 3.07,
  '101 Dalmatians (One Hundred and One Dalmatians) (1961)': 3.43,
  "101 Dalmatians II: Patch's London Adventure (2003)": 2.5,
  '101 Reykjavik (101 Re

In [ ]:
film_median = data.groupby('title').rating.agg(rating_median_film = ('median')).round(2).apply(list).to_dict()
film_median

{'rating_median_film': {"'71 (2014)": 4.0,
  "'Hellboy': The Seeds of Creation (2004)": 4.0,
  "'Round Midnight (1986)": 3.5,
  "'Salem's Lot (2004)": 5.0,
  "'Til There Was You (1997)": 4.0,
  "'Tis the Season for Love (2015)": 1.5,
  "'burbs, The (1989)": 3.5,
  "'night Mother (1986)": 3.0,
  '(500) Days of Summer (2009)': 4.0,
  '*batteries not included (1987)': 3.5,
  '...All the Marbles (1981)': 2.0,
  '...And Justice for All (1979)': 3.0,
  '00 Schneider - Jagd auf Nihil Baxter (1994)': 4.5,
  '1-900 (06) (1994)': 4.0,
  '10 (1979)': 3.5,
  '10 Cent Pistol (2015)': 1.25,
  '10 Cloverfield Lane (2016)': 4.0,
  '10 Items or Less (2006)': 3.0,
  '10 Things I Hate About You (1999)': 3.5,
  '10 Years (2011)': 3.5,
  '10,000 BC (2008)': 3.0,
  '100 Girls (2000)': 3.5,
  '100 Streets (2016)': 2.5,
  '101 Dalmatians (1996)': 3.0,
  '101 Dalmatians (One Hundred and One Dalmatians) (1961)': 3.5,
  "101 Dalmatians II: Patch's London Adventure (2003)": 2.5,
  '101 Reykjavik (101 Reykjavík) (

Создадим и заполним новые столбцы датасета средними и медианными оценками пользователей и фильмов

In [ ]:
data['rating_mean_user'] = data['userId'].apply(lambda x: user_mean['rating_mean_user'].get(x))
data

,userId,movieId,rating,year,month,title,genres,tag,rating_mean_user
0,1,1,4.0,2000,7,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar,4.0
1,1,1,4.0,2000,7,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar,4.0
2,1,1,4.0,2000,7,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,fun,4.0
3,1,3,4.0,2000,7,Grumpier Old Men (1995),Comedy|Romance,moldy,4.0
4,1,3,4.0,2000,7,Grumpier Old Men (1995),Comedy|Romance,old,4.0
...,...,...,...,...,...,...,...,...,...
285757,610,168252,5.0,2017,5,Logan (2017),Action|Sci-Fi,emotional,4.0
285758,610,168252,5.0,2017,5,Logan (2017),Action|Sci-Fi,gritty,4.0
285759,610,168252,5.0,2017,5,Logan (2017),Action|Sci-Fi,heartbreaking,4.0
285760,610,168252,5.0,2017,5,Logan (2017),Action|Sci-Fi,predictible plot,4.0


In [ ]:
data['rating_median_user'] = data['userId'].apply(lambda x: user_median['rating_median_user'].get(x))
data

,userId,movieId,rating,year,month,title,genres,tag,rating_mean_user,rating_median_user
0,1,1,4.0,2000,7,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar,4.0,4.0
1,1,1,4.0,2000,7,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar,4.0,4.0
2,1,1,4.0,2000,7,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,fun,4.0,4.0
3,1,3,4.0,2000,7,Grumpier Old Men (1995),Comedy|Romance,moldy,4.0,4.0
4,1,3,4.0,2000,7,Grumpier Old Men (1995),Comedy|Romance,old,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...
285757,610,168252,5.0,2017,5,Logan (2017),Action|Sci-Fi,emotional,4.0,4.0
285758,610,168252,5.0,2017,5,Logan (2017),Action|Sci-Fi,gritty,4.0,4.0
285759,610,168252,5.0,2017,5,Logan (2017),Action|Sci-Fi,heartbreaking,4.0,4.0
285760,610,168252,5.0,2017,5,Logan (2017),Action|Sci-Fi,predictible plot,4.0,4.0


In [ ]:
data['rating_mean_film'] = data['title'].apply(lambda x: film_mean['rating_mean_film'].get(x))
data

,userId,movieId,rating,year,month,title,genres,tag,rating_mean_user,rating_median_user,rating_mean_film
0,1,1,4.0,2000,7,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar,4.0,4.0,3.92
1,1,1,4.0,2000,7,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar,4.0,4.0,3.92
2,1,1,4.0,2000,7,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,fun,4.0,4.0,3.92
3,1,3,4.0,2000,7,Grumpier Old Men (1995),Comedy|Romance,moldy,4.0,4.0,3.26
4,1,3,4.0,2000,7,Grumpier Old Men (1995),Comedy|Romance,old,4.0,4.0,3.26
...,...,...,...,...,...,...,...,...,...,...,...
285757,610,168252,5.0,2017,5,Logan (2017),Action|Sci-Fi,emotional,4.0,4.0,4.28
285758,610,168252,5.0,2017,5,Logan (2017),Action|Sci-Fi,gritty,4.0,4.0,4.28
285759,610,168252,5.0,2017,5,Logan (2017),Action|Sci-Fi,heartbreaking,4.0,4.0,4.28
285760,610,168252,5.0,2017,5,Logan (2017),Action|Sci-Fi,predictible plot,4.0,4.0,4.28


In [ ]:
data['rating_median_film'] = data['title'].apply(lambda x: film_median['rating_median_film'].get(x))
data

,userId,movieId,rating,year,month,title,genres,tag,rating_mean_user,rating_median_user,rating_mean_film,rating_median_film
0,1,1,4.0,2000,7,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar,4.0,4.0,3.92,4.0
1,1,1,4.0,2000,7,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar,4.0,4.0,3.92,4.0
2,1,1,4.0,2000,7,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,fun,4.0,4.0,3.92,4.0
3,1,3,4.0,2000,7,Grumpier Old Men (1995),Comedy|Romance,moldy,4.0,4.0,3.26,3.0
4,1,3,4.0,2000,7,Grumpier Old Men (1995),Comedy|Romance,old,4.0,4.0,3.26,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
285757,610,168252,5.0,2017,5,Logan (2017),Action|Sci-Fi,emotional,4.0,4.0,4.28,4.5
285758,610,168252,5.0,2017,5,Logan (2017),Action|Sci-Fi,gritty,4.0,4.0,4.28,4.5
285759,610,168252,5.0,2017,5,Logan (2017),Action|Sci-Fi,heartbreaking,4.0,4.0,4.28,4.5
285760,610,168252,5.0,2017,5,Logan (2017),Action|Sci-Fi,predictible plot,4.0,4.0,4.28,4.5


Сформируем TF-IDF на тегах и жанрах

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#  преобразуем жанры фильмов в векторы

def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))


film_genres = [change_string(g) for g in data.genres.values]

tfidf = TfidfVectorizer()
film_tfidf = tfidf.fit_transform(film_genres)
# film_tfidf

df_film_tfidf = pd.DataFrame(film_tfidf.toarray(), columns=tfidf.get_feature_names_out())
df_film_tfidf

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,0.000000,0.348743,0.548508,0.527195,0.255409,0.000000,0.0,0.000000,0.484096,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
1,0.000000,0.348743,0.548508,0.527195,0.255409,0.000000,0.0,0.000000,0.484096,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
2,0.000000,0.348743,0.548508,0.527195,0.255409,0.000000,0.0,0.000000,0.484096,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.512716,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.858558,0.000000,0.000000,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.512716,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.858558,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285757,0.651005,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.759073,0.000000,0.0,0.0
285758,0.651005,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.759073,0.000000,0.0,0.0
285759,0.651005,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.759073,0.000000,0.0,0.0
285760,0.651005,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.759073,0.000000,0.0,0.0


In [ ]:
#  преобразуем теги фильмов в векторы

def change_string(s):
    return str(s).replace(' ', '').replace('-', '').lower()

tag_films = []
film = []

for movie, group in tqdm(data.groupby('title')):

    tag_films.append(' '.join([change_string(s) for s in group.tag.values]))
    film.append(movie)

tfidf_tag = TfidfVectorizer()
tag_tfidf = tfidf.fit_transform(tag_films)
# # film_tfidf

df_tag_tfidf = pd.DataFrame(tag_tfidf.toarray(), columns=tfidf.get_feature_names_out())
df_tag_tfidf

  0%|          | 0/9719 [00:00<?, ?it/s]

,06oscarnominatedbestmovieanimation,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001like,2danimation,...,worldwari,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9716,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9717,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# добавим новые признаки в датасет

df = data.loc[:, ~data.columns.isin(['title', 'genres', 'tag'])]

df = pd.merge(df, df_film_tfidf, left_index=True, right_index=True)

df = pd.merge(df, df_tag_tfidf, left_index=True, right_index=True)

df

,userId,movieId,rating,year,month,rating_mean_user,rating_median_user,rating_mean_film,rating_median_film,action_x,...,worldwari,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel
0,1,1,4.0,2000,7,4.0,4.0,3.92,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,4.0,2000,7,4.0,4.0,3.92,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,4.0,2000,7,4.0,4.0,3.92,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,3,4.0,2000,7,4.0,4.0,3.26,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,3,4.0,2000,7,4.0,4.0,3.26,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9714,21,356,4.5,2014,8,3.5,3.5,4.16,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9715,21,356,4.5,2014,8,3.5,3.5,4.16,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9716,21,356,4.5,2014,8,3.5,3.5,4.16,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9717,21,356,4.5,2014,8,3.5,3.5,4.16,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Построим модель линейной регрессии

In [ ]:
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics

Поместим признаки в переменную X, а целевую переменную — в y.

In [ ]:
X = df.iloc[:, 3:]
X

,year,month,rating_mean_user,rating_median_user,rating_mean_film,rating_median_film,action_x,adventure_x,animation_x,children_x,...,worldwari,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel
0,2000,7,4.0,4.0,3.92,4.0,0.0,0.348743,0.548508,0.527195,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2000,7,4.0,4.0,3.92,4.0,0.0,0.348743,0.548508,0.527195,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2000,7,4.0,4.0,3.92,4.0,0.0,0.348743,0.548508,0.527195,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2000,7,4.0,4.0,3.26,3.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2000,7,4.0,4.0,3.26,3.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9714,2014,8,3.5,3.5,4.16,4.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9715,2014,8,3.5,3.5,4.16,4.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9716,2014,8,3.5,3.5,4.16,4.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9717,2014,8,3.5,3.5,4.16,4.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#

array([[-0.83625816,  0.09346896,  0.52355622, ..., -0.01014405,
        -0.02268746, -0.01014405],
       [-0.83625816,  0.09346896,  0.52355622, ..., -0.01014405,
        -0.02268746, -0.01014405],
       [-0.83625816,  0.09346896,  0.52355622, ..., -0.01014405,
        -0.02268746, -0.01014405],
       ...,
       [ 0.89280674,  0.39810504, -0.07557753, ..., -0.01014405,
        -0.02268746, -0.01014405],
       [ 0.89280674,  0.39810504, -0.07557753, ..., -0.01014405,
        -0.02268746, -0.01014405],
       [ 0.89280674,  0.39810504, -0.07557753, ..., -0.01014405,
        -0.02268746, -0.01014405]])

In [ ]:
y = df['rating']
y

,rating
0,4.0
1,4.0
2,4.0
3,4.0
4,4.0
...,...
9714,4.5
9715,4.5
9716,4.5
9717,4.5


Разделим выборку на тестовую и обучающую

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
X_train.shape # посмотрим размер обучающей выборки

(7775, 1496)

In [ ]:
y_train.shape

(7775,)

In [ ]:
X_test.shape # посмотрим размер тестовой выборки

(1944, 1496)

In [ ]:
y_test.shape

(1944,)

In [ ]:
linear_model = LinearRegression()

In [ ]:
linear_model.fit(X_train, y_train) # метод обучается на данных и подбирает оптимальные коэффициенты

LinearRegression()

In [ ]:
linear_model.coef_ # посмотрим, какие параметры достигла модель

array([-0.0069017 , -0.03774832,  0.31654379, ...,  0.        ,
       -0.34735262,  0.        ])

In [ ]:
linear_model.intercept_

13.251752584987605

In [ ]:
y_pred = linear_model.predict(X_test) # тестируем получившуюся модель (передаем значение X и выводим значение y, которые предсказала модель на этих X)
y_pred

array([3.93444023, 3.75775458, 1.82715924, ..., 3.57514934, 3.26621593,
       3.63796608])

In [ ]:
y_test # реальные ответы X, которые были собраны при реальных наблюдениях

,rating
251,5.0
4961,4.0
3465,1.0
2755,4.0
6963,3.0
...,...
6650,4.0
5072,4.0
4993,2.0
1561,2.0


In [ ]:
y_pred1 = linear_model.predict(X_train) # проверим метрики для обучающего множества
y_pred1

array([4.32694055, 4.1578087 , 2.92938187, ..., 3.8444095 , 3.62670106,
       4.08127862])

In [ ]:
y_train

,rating
7609,4.0
1739,5.0
2098,2.0
8948,1.0
2373,3.0
...,...
5734,5.0
5191,3.0
5390,4.0
860,5.0


Оценим качество модели при помощи сравнения предсказанных ответов X модели и реальных ответов X (собранных при реальных наблюдениях).

In [ ]:
linear_model.score(X_test, y_test) # метод возвращает значение коэффициента детерминированности

0.35636058314933616

In [ ]:
linear_model.score(X_train, y_train)

0.4277390085903582

Оценим модель

In [ ]:
print(f'R2_test: {metrics.r2_score(y_test, y_pred)}') # предпочтительны модели с более высокими значениями R-квадрата, поскольку это означает, что набор переменных-предикторов в модели способен хорошо объяснить изменение переменной отклика
print(f'RMSE_test: {np.sqrt(metrics.mean_squared_error(y_test, y_pred))}') # cреднеквадратическая ошибка показывает, насколько далеко в среднем прогнозируемые значения от наблюдаемых значений в модели (Чем больше RMSE, тем больше разница между прогнозируемыми и наблюдаемыми значениями, а это означает, что модель хуже соответствует данным)
print(f'R2_train: {metrics.r2_score(y_train, y_pred1)}')
print(f'RMSE_train: {np.sqrt(metrics.mean_squared_error(y_train, y_pred1))}')

R2_test: 0.35636058314933616
RMSE_test: 0.918984681646991
R2_train: 0.4277390085903582
RMSE_train: 0.8637938336742103


Точность модели невысокая, так как коэффициент детерминированности невелик = 35.63%, разница между прогнозируемыми и наблюдаемыми значениями велика, о чем свидетельствует величина cреднеквадратической ошибки.

В идеале R2 = 1, а RMSE = 0.

**Вывод:** полученная невысокая точность модели объясняется небольшим размером набора данных и тестовой выборки, а также небольшим количеством признаков с ненулевыми значениями в итоговом датасете (признаки, полученные при формировании TF-IDF на тегах и жанрах).

Для повышения точности модели целесообразно формировать примерно равный по числу значений размер выборок по различным признакам.